# Accessibility of London foodbanks

## Get foodbanks

1. Independent foodbanks: Data from list published in the [Guardian](https://www.theguardian.com/society/2017/may/29/report-reveals-scale-of-food-bank-use-in-the-uk-ifan). 
2. Trussell Trust foodbanks listed on their [website](https://www.trusselltrust.org/get-help/find-a-foodbank/)

In [19]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import geopandas as gpd
from shapely.geometry import Point, Polygon

### Independent foodbanks

In [110]:
#clean up raw data
ind_raw = pd.read_csv("IFAN_members_London.csv", header=None)
ind_data = {"Name":[], "Postcode":[]}

for i in np.arange(0, len(ind_raw), 4):
    ind_data["Name"].append(ind_raw.iloc[i+1].values[0])
    ind_data["Postcode"].append(ind_raw.iloc[i+3].values[0])

ind = pd.DataFrame.from_dict(ind_data)

In [111]:
# get coordinates for foodbanks using Nominatim API and postcode
app = Nominatim(user_agent="mapping_projects")

geodata = {"lat": [], "lon": []}
for location in ind["Postcode"]:
    try:
        loc_data = app.geocode(location).raw
        geodata["lat"].append(loc_data["lat"])
        geodata["lon"].append(loc_data["lon"])
    except BaseException:
        geodata["lat"].append("Missing")
        geodata["lon"].append("Missing")                     

ind["lat"] = geodata["lat"]
ind["lon"] = geodata["lon"]

ind.to_csv("IFAN-members-london-final.csv")

### Trussell Trust members

In [20]:
#clean up raw data
trussell_raw = pd.read_excel("foodbanks-trussell.xlsx", header=None)
trussell_raw = trussell_raw.dropna()
trussell_raw = trussell_raw.loc[~trussell_raw[0].str.contains("location")]
trussell_raw = trussell_raw.loc[~trussell_raw[0].str.contains("details")]
trussell_raw = trussell_raw.loc[~trussell_raw[0].str.contains("from London")]
trussell_raw = trussell_raw.reset_index(drop = True)

In [61]:
data = {"Name":[], "Address":[]}

for i in np.arange(0, len(trussell_raw), 2):
    data["Name"].append(trussell_raw.iloc[i].values[0])
    data["Address"].append(trussell_raw.iloc[i+1].values[0])

trussell_cleaned = pd.DataFrame.from_dict(data)

In [62]:
values = {", UK": "", ", United Kingdom": ""}

for string in values.items():
    trussell_cleaned["Address"] = trussell_cleaned["Address"].str.replace(string[0], string[1])

In [ ]:
# get coordinates for region using Nominatim API
app = Nominatim(user_agent="mapping_projects")

geodata = {"lat": [], "lon": []}
for location in trussell["Address"]:
    try:
        loc_data = app.geocode(location).raw
        geodata["lat"].append(loc_data["lat"])
        geodata["lon"].append(loc_data["lon"])
    except BaseException:
        geodata["lat"].append("Missing")
        geodata["lon"].append("Missing")                     

trussell_cleaned["lat"] = geodata["lat"]
trussell_cleaned["lon"] = geodata["lon"]

trussell_cleaned.to_csv("trussell_cleaned.csv")

Did a lot more offline cleaning on this one ...

### save final list of foodbanks as shapefile

In [133]:
### Foodbanks
ind = pd.read_csv("IFAN-members-london-final.csv")
trussell = pd.read_csv("trussell_final.csv")
ind = ind.rename(columns={"Postcode":"Address"})
ind["Type"] = "Independent"
trussell["Type"] = "Trussell Trust"

#merge both datasets
df = pd.concat([ind, trussell], ignore_index=True)

#turn into geodataframe
geometry = [Point(xy) for xy in zip(df['lon'].astype(float), df['lat'].astype(float))]
gdf = gpd.GeoDataFrame(df, crs = 'EPSG:4326', geometry = geometry)
gdf.head()

,Name,Address,lat,lon,Type,geometry
0,St Alban's Foodbank,SE25 6RD,51.39822,-0.09156,Independent,POINT (-0.09156 51.39822)
1,South Norwood Community Kitchen,SE25 6TY,51.40005,-0.08001,Independent,POINT (-0.08001 51.40005)
2,St Alban's Foodbank,SE25 6RD,51.39822,-0.09156,Independent,POINT (-0.09156 51.39822)
3,Liberty Food Bank,SW16 4JS,51.41027,-0.12052,Independent,POINT (-0.12052 51.41027)
4,South Norwood Community Kitchen,SE25 6TY,51.40005,-0.08001,Independent,POINT (-0.08001 51.40005)


In [135]:
#filter out those outside of London
la = gpd.GeoDataFrame.from_file("London_Borough_Excluding_MHW.geojson")
la = la.to_crs(4326)
gdf = gpd.sjoin(gdf, la, how="inner")
gdf = gdf.reset_index(drop=True)

In [139]:
gdf.to_file("foodbanks-london-final.geojson", driver='GeoJSON')

# Other

## Scraping - failed

In [23]:
#request = requests.get("https://www.foodaidnetwork.org.uk/our-members")
url = "https://www.trusselltrust.org/get-help/find-a-foodbank/"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0'}
result = requests.get(url, headers=headers)

#russell = requests.json()

In [ ]:
"https://interactive.guim.co.uk/tools/tables/embed/index.html?spreadsheet=1HwmTt1GWBlyLbDt_lT2yrQyLSat1z-TQEE-X2MKQS9M"

In [24]:
soup = BeautifulSoup(result.content, 'html.parser')

In [26]:
soup

<!DOCTYPE html>

<html lang="en-GB">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<title>Find a Food Bank - The Trussell Trust</title>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="https://www.trusselltrust.org/xmlrpc.php" rel="pingback"/>
<!-- Critical CSS -->
<style>
		.btn {background-repeat: repeat!important;}
	</style>
<!-- LoadCSS (stylesheets after the fact) -->
<script>
		function loadCSS(e,o,n,s){"use strict";var t=window.document.createElement("link"),l=o||window.document.getElementsByTagName("script")[0],c=window.document.styleSheets;return t.rel="stylesheet",t.href=e,t.media="only x",s&&(t.onload=s),l.parentNode.insertBefore(t,l),t.onloadcssdefined=function(o){for(var n,s=0;s<c.length;s++)c[s].href&&c[s].href.indexOf(e)>-1&&(n=!0);n?o():setTimeout(function(){t.onloadcssdefined(o)})},t.onloadcssdefined(function(){t.media=n||"screen, print"}),t}
		l

In [13]:
request = requests.get("https://www.trusselltrust.org/")